In [1]:
from IPython.core.display import display, HTML
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

C:\Users\adidr\AppData\Local\Temp\ipykernel_35296\233690474.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Goals
1. Subscribing to real time market data
2. Using pendingTickersEvent to handle updates
3. Display the watchlist!

### Connect to IB

In [3]:
from ib_insync import *
util.startLoop()
ib = IB()
ib.connect(port=4001,clientId=10)

<IB connected to 127.0.0.1:4001 clientId=10>

In [4]:
a=[1,2,3]
a[::-1]

[3, 2, 1]

### Subscribe to Market Data

In [5]:
contract = Stock(symbol='TSLA',exchange='SMART',currency='USD')
ib.qualifyContracts(contract)
ticker = ib.reqMktData(contract,"",False,False)

In [6]:
 ticker.high

nan

### Use PendingTickersEvent to handle ticker updates

In [7]:
def onPendingTickers(tickers):
    for ticker in tickers:
        print(ticker.contract.symbol,ticker.bid,ticker.bidSize,ticker.ask,ticker.askSize,ticker.last,ticker.lastSize)

### Cancel market data 

In [8]:
ib.cancelMktData(contract)

### Symbols

In [9]:
symbol_list = ['TSLA','RIVN','QQQ','BABA','NVDA','META','AAPL']

In [10]:
for symbol in symbol_list:
    contract = Stock(symbol,exchange='SMART',currency='USD')
    ib.qualifyContracts(contract)
    ticker = ib.reqMktData(contract,"",False,False)

In [11]:
ib.pendingTickersEvent.clear()

In [12]:
previous_trade_prices={symbol:-1 for symbol in symbol_list}
previous_colors={symbol:'' for symbol in symbol_list}
def color_row(row):
    color = previous_colors[row.name]
    if row.name in previous_trade_prices:
       if row['lastTradePrice']>previous_trade_prices[row.name]:
           color = 'green'
       elif row['lastTradePrice']<previous_trade_prices[row.name]:
           color = 'red'
       else:
           color = previous_colors[row.name]
    
    style = f'background-color: {color}'
    previous_trade_prices[row.name]=row['lastTradePrice']
    previous_colors[row.name]=color
    return [style]*len(row)

In [13]:
import pandas as pd 
import numpy as np
from IPython.display import display,clear_output
df = pd.DataFrame(columns=['bid','bidSize','ask','askSize','lastTradePrice','lastTradeVolume','high','close','PCT_CHANGE'],index=symbol_list)
def onPendingTickersWithStyle(tickers):
    for ticker in list(tickers)[::-1]:
        df.loc[ticker.contract.symbol]=(ticker.bid,ticker.bidSize,ticker.ask,ticker.askSize,ticker.last,ticker.lastSize,ticker.high,ticker.close,np.round((ticker.last-ticker.close)/ticker.close*100,2))
    styled_df = df.style.apply(color_row, axis=1).set_table_styles([{'selector': 'th',
                                          'props': [('background-color', '#ffff99'),
                                                    ('font-size', '16px')]},  # Make header font larger
                                         {'selector': 'td',
                                          'props': [('font-size', '16px')]},  # Make cell font larger
                                         {'selector': 'td:hover',
                                          'props': [('background-color', 'cyan')]},  # Highlight cells on hover
                                         ],
                                        overwrite=False)
    clear_output()
    display(styled_df)

In [14]:
ib.pendingTickersEvent.clear()

In [15]:
ib.pendingTickersEvent+=onPendingTickersWithStyle

,bid,bidSize,ask,askSize,lastTradePrice,lastTradeVolume,high,close,PCT_CHANGE
TSLA,nan,nan,nan,nan,nan,nan,nan,nan,nan
RIVN,10.970000,3200.000000,10.980000,6800.000000,10.970000,200.000000,nan,11.450000,-4.190000
QQQ,439.800000,800.000000,439.810000,300.000000,439.800000,100.000000,nan,438.070000,0.390000
BABA,76.300000,200.000000,76.350000,200.000000,76.310000,100.000000,nan,76.120000,0.250000
NVDA,804.600000,500.000000,804.750000,200.000000,804.750000,500.000000,nan,785.380000,2.470000
META,nan,nan,nan,nan,nan,nan,nan,nan,nan
AAPL,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [17]:
df.style.apply(color_row,axis=1)

,bid,bidSize,ask,askSize,lastTradePrice,lastTradeVolume,high,close,PCT_CHANGE
TSLA,192.270000,100.000000,192.290000,1100.000000,192.280000,100.000000,197.570000,197.410000,-2.600000
RIVN,10.220000,103200.000000,10.230000,126400.000000,10.230000,100.000000,11.020000,11.450000,-10.660000
QQQ,436.960000,700.000000,436.970000,1200.000000,436.960000,100.000000,440.590000,438.070000,-0.250000
BABA,75.920000,500.000000,75.930000,4200.000000,75.920000,100.000000,76.930000,76.120000,-0.260000
NVDA,789.880000,300.000000,790.090000,100.000000,789.980000,100.000000,823.940000,785.380000,0.590000
META,483.840000,300.000000,483.910000,400.000000,483.890000,100.000000,494.360000,486.130000,-0.460000
AAPL,182.590000,5800.000000,182.600000,900.000000,182.590000,100.000000,185.040000,184.370000,-0.970000


###### 